In [9]:
import numpy as np
import os
import pickle
from tqdm import tqdm
from pathlib import Path
from collections import Counter

In [2]:
array = np.array([[1, 2, 3], [4, 5, 6]])

In [8]:
Path(os.path.realpath("/mnt/File/my_file/2024-2/fed-multimodal/fed_multimodal/output/log/fed_avg/uci-har")).parents[1]

PosixPath('/mnt/File/my_file/2024-2/fed-multimodal/fed_multimodal/output/log')

In [10]:
Counter([1,2,2,3,3,3])

Counter({1: 1, 2: 2, 3: 3})